In [1]:
import pandas as pd
import numpy as np
import openrouteservice as ors
from dotenv import load_dotenv
from pyonemap import OneMap
import os
import requests
import json
import time
from collections import namedtuple
load_dotenv()
import folium
import plotly.express as px

In [3]:
#get mrt data
mrt_df = pd.read_csv('../data/mrt_station_final.csv')
print(mrt_df.head())
coords = mrt_df[['Latitude', 'Longitude']].values.tolist()
names = mrt_df['MRT.Name']
mrt_lat = mrt_df['Latitude']
mrt_lon = mrt_df['Longitude']
# run this to start OTP: java -Xmx6G -jar otp-2.5.0-shaded.jar --load C:\Users\wjlwi\OneDrive\Desktop\Github\Public-Transportation-In-Singapore\Backend\otp  --serve
#java -Xmx6G -jar otp-2.5.0-shaded.jar --load /Users/ethan/Documents/GitHub/Public-Transportation-In-Singapore/Backend/otp/  --serve

# Sample API request to the OTP isochrone endpoint
response = requests.get("http://localhost:8080/otp/traveltime/isochrone?batch=true&location=1.3521,103.895&time=2023-04-12T10:19:03%2B02:00&modes=WALK,TRANSIT&arriveBy=false&cutoff=30M17S")
# isochrone_geojson = response.json()

   S/N                 MRT.Name  Latitude   Longitude
0    1    ESPLANADE MRT STATION  1.293658  103.855081
1    2   PAYA LEBAR MRT STATION  1.317199  103.892365
2    3  DHOBY GHAUT MRT STATION  1.298655  103.846194
3    4       DAKOTA MRT STATION  1.308548  103.889065
4    5     LAVENDER MRT STATION  1.307378  103.862768


Write a function for fetching isochrome. Configure params here

In [4]:
def fetch_isochrone(lat, lon, modes, cutoff="20M17S"):
    """Fetch isochrone GeoJSON for a given location."""
    url = "http://localhost:8080/otp/traveltime/isochrone"
    params = {
        "batch": "true",
        "location": f"{lat},{lon}",
        "time": "2023-04-12T10:19:03+02:00",
        "modes": modes,
        "arriveBy": "false",
        "cutoff": cutoff
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch isochrone for {lat},{lon}. Status code: {response.status_code}")
        return None

x = fetch_isochrone(1.293657725, 103.8550812, modes = "TRANSIT")
features = x['features'][0]['geometry']['coordinates'][0][0]
print(features)


[[103.845918, 1.288278], [103.847592, 1.289179], [103.847909, 1.28847], [103.848145, 1.288153], [103.848729, 1.287492], [103.849391, 1.287183], [103.849732, 1.286695], [103.850084, 1.286355], [103.850453, 1.285618], [103.85119, 1.284884], [103.851404, 1.28477], [103.852441, 1.284556], [103.85286, 1.284427], [103.852989, 1.284384], [103.854273, 1.284041], [103.854788, 1.28334], [103.855225, 1.284556], [103.855393, 1.285161], [103.855724, 1.286355], [103.856587, 1.287227], [103.857917, 1.287684], [103.858386, 1.287509], [103.858984, 1.286952], [103.858932, 1.286355], [103.859228, 1.285397], [103.860185, 1.285903], [103.860735, 1.286355], [103.861985, 1.287267], [103.863722, 1.288091], [103.863784, 1.287198], [103.86493, 1.288153], [103.865289, 1.289658], [103.865363, 1.289952], [103.865093, 1.291262], [103.865078, 1.291751], [103.864972, 1.292939], [103.863784, 1.293382], [103.8637, 1.293466], [103.863669, 1.293549], [103.86307, 1.294635], [103.86262, 1.295348], [103.863298, 1.296661], [

In [5]:
import plotly.graph_objs as go
import random

f = go.FigureWidget()
f.layout.hovermode = 'closest'
f.layout.hoverdistance = -1 #ensures no "gaps" for selecting sparse data
default_linewidth = 2
highlighted_linewidth_delta = 2

# just some traces with random data points  
num_of_traces = 5
random.seed = 42
for i in range(num_of_traces):
    y = [random.random() + i / 2 for _ in range(100)]
    trace = go.Scatter(y=y, mode='lines', line={ 'width': default_linewidth })
    f.add_trace(trace)

# our custom event handler
def update_trace(trace, points, selector):
    # this list stores the points which were clicked on
    # in all but one trace they are empty
    if len(points.point_inds) == 0:
        return
    for i,_ in enumerate(f.data):
        f.data[i]['line']['width'] = default_linewidth + highlighted_linewidth_delta * (i == points.trace_index)
    print(points.trace_index)

# we need to add the on_click event to each trace separately       
for i in range(len(f.data) ):
    f.data[i].on_click(update_trace)

# let's show the figure 
f

FigureWidget({
    'data': [{'line': {'width': 2},
              'mode': 'lines',
              'type': 'scatter',
              'uid': 'fd28f6ab-3690-42b9-bb22-5182e65310c9',
              'y': [0.8954805677660957, 0.2913982057288046, 0.19173787575896428,
                    0.420310264347331, 0.16871455900107368, 0.6618342666864375,
                    0.3779819207340057, 0.5263592240262186, 0.6377569218630659,
                    0.884915271245198, 0.2639361706519828, 0.03172648286363822,
                    0.8665690267665805, 0.5371604223428879, 0.044389209182576495,
                    0.5728349184418755, 0.02404949299181547, 0.0653380740339482,
                    0.735337012603764, 0.23779571179619963, 0.11911605312282558,
                    0.08062962016501696, 0.9580902377730278, 0.5397452455543975,
                    0.15709140669257027, 0.9104454343058469, 0.53393262171068,
                    0.11821362330444252, 0.6429805512163338, 0.9045615550916208,
                  

1
4
3


In [42]:
import plotly.graph_objects as go


fig = go.Figure(go.Scattermapbox(lat=[36,37,38],
                                 lon=[26,27,28],
                                 mode='markers',
                                 marker={'size':9,'color':'purple'},
                                 selected=go.scattermapbox.Selected(marker = {"color":"red", "size":25})
                                 )
                )

fig.update_layout(clickmode='event+select',
                  mapbox = {'style': "stamen-terrain",
                            'center': {'lon': 27.0, 'lat': 37.0 },
                            'zoom': 6},
                  margin = dict(l=0, r=0, t=0, b=0)
                  )

fig.show()

In [51]:
from plotly.subplots import make_subplots
f = go.FigureWidget(make_subplots(rows=2, cols=1))

trace1 = go.Scatter(x=[3, 4, 5], y=[10, 11, 12], name='trae1')
trace2 = go.Scatter(x=[6, 7, 8], y=[13, 14, 15])

f.add_trace(trace1, row=1, col=1)
f.add_trace(trace2, row=2, col=1)

def print_coordinates(trace, points, selector):
    if points.point_inds:  # Ensure a point is clicked
        # Extract the coordinates of the clicked marker
        print(trace)
        clicked_lon = trace.x[1]
        clicked_lat = trace.y[1]
        # Print the coordinates
        print(f"Clicked marker coordinates: Lat {clicked_lat}, Lon {clicked_lon}")

for trace in f.data:
    trace.on_click(print_coordinates)
f

FigureWidget({
    'data': [{'name': 'trae1',
              'type': 'scatter',
              'uid': '342389d7-2a00-41bb-8905-d0510cf69870',
              'x': [3, 4, 5],
              'xaxis': 'x',
              'y': [10, 11, 12],
              'yaxis': 'y'},
             {'type': 'scatter',
              'uid': '8366327d-35a8-41b7-b31d-7674fdc993ca',
              'x': [6, 7, 8],
              'xaxis': 'x2',
              'y': [13, 14, 15],
              'yaxis': 'y2'}],
    'layout': {'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0]},
               'xaxis2': {'anchor': 'y2', 'domain': [0.0, 1.0]},
               'yaxis': {'anchor': 'x', 'domain': [0.575, 1.0]},
               'yaxis2': {'anchor': 'x2', 'domain': [0.0, 0.425]}}
})

Scatter({
    'name': 'trae1',
    'uid': '342389d7-2a00-41bb-8905-d0510cf69870',
    'x': [3, 4, 5],
    'xaxis': 'x',
    'y': [10, 11, 12],
    'yaxis': 'y'
})
Clicked marker coordinates: Lat 11, Lon 4


Sample to Edit for Hovering

In [53]:
import plotly.graph_objects as go
# Create a Plotly figure
fig = go.FigureWidget()
fig.layout.hovermode = 'closest'
default_linewidth = 2
highlighted_linewidth = 3
color_sequence = ['blue', 'green', 'red', 'orange', 'purple', 'yellow']

def print_coordinates(trace, points, selector):
    print("Hi")

# Fetch and add isochrones for each MRT station
for index, (lat, lon) in enumerate(coords):
    mrt_name = mrt_df[(mrt_df['Latitude'] == lat) & (mrt_df['Longitude'] == lon)]['MRT.Name'].iloc[0]
    # Add isochrone as a trace
    color = color_sequence[index%len(color_sequence)]
    fig.add_trace(go.Scattermapbox(
        mode="markers",
        lon=[lon],
        lat=[lat],
        text = [mrt_name],
        marker=dict(size=10,color=color),
        selected=go.scattermapbox.Selected(marker = {"size":25}),
        unselected=go.scattermapbox.Unselected(marker = {"color":"grey", "size":5})
    ))
for trace in fig.data:
    trace.on_click(print_coordinates)
# Define layout for the map
fig.update_layout(
    clickmode='event+select',
    mapbox_style="carto-positron",
    mapbox_zoom=12,
    mapbox_center={"lat": coords[0][0], "lon": coords[0][1]},
    showlegend = False
)
fig.show()

Static Map using go.Figure()

In [111]:
import plotly.graph_objects as go
# Create a Plotly figure
fig = go.Figure()
color_sequence = ['blue', 'green', 'red', 'orange', 'purple', 'yellow'] 
# Fetch and add isochrones for each MRT station
for index, (lat, lon) in enumerate(coords):
    isochrone_geojson = fetch_isochrone(lat, lon, modes="TRANSIT")
    mrt_name = mrt_df[(mrt_df['Latitude'] == lat) & (mrt_df['Longitude'] == lon)]['MRT.Name'].iloc[0]
    if isochrone_geojson and 'features' in isochrone_geojson:
        # Access the first feature in the returned GeoJSON
        feature = isochrone_geojson['features'][0]
        if 'geometry' in feature:
            geometry = feature['geometry']
            if 'coordinates' in geometry:
                coordinates = geometry['coordinates'][0][0]
                # Assuming coordinates are in [lon, lat] format
                lon_coords = [coord[0] for coord in coordinates]
                lat_coords = [coord[1] for coord in coordinates]
                # Add isochrone as a trace
                color = color_sequence[index%len(color_sequence)]
                fig.add_trace(go.Scattermapbox(
                    mode="lines",
                    lon=lon_coords,
                    lat=lat_coords,
                    name=f"Isochrone {lat}, {lon}",
                    line=dict(width=1, color=color),
                    fill = "toself",
                    marker=dict(size=0),
                    text=[mrt_name],  # Set text for hover
                    hoverinfo='text'
                ))
                fig.add_trace(go.Scattermapbox(
                    mode="markers",
                    lon=[lon],
                    lat=[lat],
                    text=[mrt_name],
                    name=mrt_name,
                    marker=dict(size=10,color=color),
                    textposition="bottom center",
                ))


# Define layout for the map
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=12,
    mapbox_center={"lat": coords[0][0], "lon": coords[0][1]},
    showlegend = False
)

# Show the map
fig.show()

Visualise all isochrones

In [17]:

m = folium.Map(location=coords[0], zoom_start=12)

# Fetch and add isochrones for each MRT station
for lat, lon in coords:
    isochrone_geojson = fetch_isochrone(lat, lon, modes = "TRANSIT")
    if isochrone_geojson:
        folium.GeoJson(isochrone_geojson, name=f"Isochrone {lat}, {lon}").add_to(m)

# Add layer control and display the map
folium.LayerControl().add_to(m)
m

In [ ]:
#1 cycling range
#2 walking range
#3 bus + walking range
#https://github.com/gis-ops/routingpy

Saving isochrones for bus, walking, cycling.

In [ ]:
# Assuming mrt_df is already defined
mrt_df['bus 30M isochrone'] = mrt_df.apply(lambda row: fetch_isochrone(row['Latitude'], row['Longitude'],modes = "BUS,WALK", cutoff="20M17S"), axis=1) #change modes to BUS/WALK
mrt_df['walking 30M isochrone'] = mrt_df.apply(lambda row: fetch_isochrone(row['Latitude'], row['Longitude'],modes = "WALK", cutoff="20M17S"), axis=1) #change modes to WALK
mrt_df['cycling 30M isochrone'] = mrt_df.apply(lambda row: fetch_isochrone(row['Latitude'], row['Longitude'],modes = "BICYCLE", cutoff="20M17S"), axis=1) #change modes to CYCLE


save data as .pkl to preserve json

In [ ]:
#mrt_df.to_pickle("../data/mrt_df_with_isochrones.pkl")
mrt_df = pd.read_pickle("../data/mrt_df_with_isochrones.pkl")

In [ ]:
import folium

m = folium.Map(location=[1.3521, 103.8198], zoom_start=12)

for index, row in mrt_df.iterrows():
    # Creating a popup message with a brief description or link
    popup_message = f"MRT Station: {index}<br>"
    popup_message += "Cycling 30M Isochrone available."  # Example message; customize as needed
    
    # Create and add a marker to the map
    folium.Marker(
        [row['Latitude'], row['Longitude']],
        popup=folium.Popup(popup_message, parse_html=True, max_width=450)
    ).add_to(m)
m


